In [1]:
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ZeroPadding2D, UpSampling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import backend as K

import math
import matplotlib.pyplot as plt

import pypianoroll
import numpy as np
from pypianoroll import Multitrack, Track

In [2]:
# run params
run_id = '3020'

RUN_FOLDER = 'run/'
RUN_FOLDER += run_id

# Number of timestept the slices Pianorolls should have (Needs to be dividable by 16)
pianrollLength = 128

store_folder = os.path.join(RUN_FOLDER, 'store')
weights_folder = os.path.join(RUN_FOLDER, 'weights')

if not os.path.exists(RUN_FOLDER):
    os.makedirs(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'store'))
    os.mkdir(os.path.join(RUN_FOLDER, 'output'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

# Number of different Notes between highest and lowest Note
minNumDifferentNoted = 5
# Set Note bounds for faster training
lowestNotePossible = 20
highestNotePossible = 108
# possibleNotes mus be dividable by 4 else the Architekture needs to be changed
possibleNotes = highestNotePossible - lowestNotePossible

# Load Training Data

In [3]:
# Load an already shaped array
reshaped = np.load('data/preprocessed/midi_normalized_p128_dn88.npy')
reshaped.shape

(682, 128, 88, 1)

# Creating the neural Network

In [4]:
from keras.utils.generic_utils import get_custom_objects

@tf.function
def custom_activation(x):
    return (K.sigmoid(x) * K.sigmoid(x))

get_custom_objects().update({'custom_activation': Activation(custom_activation)})

# Quelle: https://stackoverflow.com/a/43915483/9179624)

Using TensorFlow backend.


In [5]:
discriminator = Sequential()

discriminator.add(Conv2D(64, kernel_size = (5,5), padding = 'same', input_shape=(pianrollLength,possibleNotes,1)))
#discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(alpha=0.3))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(64, kernel_size = (5,5), padding = 'same'))
discriminator.add(LeakyReLU(alpha=0.3))
discriminator.add(Flatten())
discriminator.add(Dropout(0.3))
discriminator.add(Dense(128, activation='sigmoid'))
discriminator.add(Dropout(0.2))
discriminator.add(Dense(1, activation='sigmoid'))

In [6]:

discriminator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 88, 64)       1664      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 88, 64)       0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 88, 64)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 88, 64)       102464    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 88, 64)       0         
_________________________________________________________________
flatten (Flatten)            (None, 720896)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 720896)            0

In [7]:
z_dim = 100

generator_initial_dense_layer_size = (int(pianrollLength/4),int(possibleNotes/4),64)

generator = Sequential()
generator.add(Dense(np.prod(generator_initial_dense_layer_size), input_shape=(z_dim,)))
generator.add(Reshape(generator_initial_dense_layer_size))
#generator.add(Dropout(0.2))
#generator.add(BatchNormalization())

generator.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear'))
generator.add(Conv2D(16, kernel_size = (5,5), padding='same'))
#generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.3))
#generator.add(Dropout(0.2))

generator.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear'))
generator.add(Conv2D(32, kernel_size = (5,5), padding='same'))
#generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.3))
#generator.add(Dropout(0.2))
 
generator.add(Conv2DTranspose(64, kernel_size = (5,5), padding = 'same'))
#generator.add(Conv2D(8, kernel_size = (5,5), padding='same'))
#generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.3))
#generator.add(Dropout(0.2))

generator.add(Conv2D(filters = 1, kernel_size = (5,5), activation=custom_activation, padding = 'same'))

In [8]:
generator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 45056)             4550656   
_________________________________________________________________
reshape (Reshape)            (None, 32, 22, 64)        0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 64, 44, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 44, 16)        25616     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 44, 16)        0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 128, 88, 16)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 88, 32)      

In [9]:
optimizer_d = RMSprop(lr=0.0001, clipvalue=1.0, decay=6e-8)

discriminator.trainable = True
discriminator.compile(optimizer=optimizer_d, loss="binary_crossentropy", metrics=["accuracy"])


optimizer_g = RMSprop(lr=0.0001, clipvalue=1.0, decay=3e-8)
# Fürs GAN soll nur der Generator trainiert werden
discriminator.trainable = False

GANModel = Sequential()
GANModel.add(generator)
GANModel.add(discriminator)
GANModel.compile(optimizer=optimizer_g, loss="binary_crossentropy", metrics=["accuracy"])

In [10]:
GANModel.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 128, 88, 1)        4641969   
_________________________________________________________________
sequential (Sequential)      (None, 1)                 92379073  
Total params: 97,021,042
Trainable params: 4,641,969
Non-trainable params: 92,379,073
_________________________________________________________________


In [11]:
GANModel.save(os.path.join(weights_folder, 'GANModel_init.h5'))
discriminator.save(os.path.join(weights_folder, 'discriminator_init.h5'))
generator.save(os.path.join(weights_folder, 'generator_init.h5'))

# Train the Model

In [12]:
#GANModel.load_weights(os.path.join(weights_folder, 'GANModel.h5'))
#discriminator.load_weights(os.path.join(weights_folder, 'discriminator.h5'))
#generator.load_weights(os.path.join(weights_folder, 'generator.h5'))

In [13]:
def train_custom(epochs, batch_size=128):
    # Save the Generator and discriminator models
    #save_model(generator, os.path.join(RUN_FOLDER, 'images/generator'))
    #save_model(discriminator, os.path.join(RUN_FOLDER, 'images/discriminator'))
    #save_model(GANModel, os.path.join(RUN_FOLDER, r"images/GANModel"))
    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    print(valid.shape)
    
    # Some variables for validations
    d_loss_real_list = []
    d_acc_real_list = []
    d_loss_fake_list = []
    d_acc_fake_list = []
    best_loss = 0.5
    g_loss = [100, 0]
    d_loss_real, d_acc_real = 99, 0 
    d_loss_fake, d_acc_fake = 99, 0
    d_loss = 99
    d_acc = 0
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------      
        current_discriminator_accuracy = 0
        # Trains the Discriminator until it reached an accuracy of over 80% on the fake images
        # If the max. accuracy is higher the generator needs to learn more to reach its min. accuracy
        while current_discriminator_accuracy < 1:
            noise = np.random.normal(0, 1, (batch_size, z_dim))
            gen_imgs = generator.predict(noise)
            # Select a random half of images
            idx = np.random.randint(0, reshaped.shape[0], batch_size)
            imgs = reshaped[idx]
            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real, d_acc_real =  discriminator.train_on_batch(imgs, valid)
            d_loss_fake, d_acc_fake =  discriminator.train_on_batch(gen_imgs, fake)
            d_loss =  0.5 * (d_loss_real + d_loss_fake)
            d_acc = 0.5 * (d_acc_real + d_acc_fake)
            current_discriminator_accuracy = d_acc
            print ("%d [D loss: (%.3f)(R %.3f, F %.3f)] [D acc: (%.3f)(%.3f, %.3f)]" % (epoch, d_loss, d_loss_real, d_loss_fake, d_acc, d_acc_real, d_acc_fake))


        # ---------------------
        #  Train Generator
        # ---------------------
        current_generator_accuracy = 0
        # Trains the generator, until it reaches an accuracy of 100%
        while current_generator_accuracy < 1:
            noise = np.random.normal(0, 1, (batch_size, z_dim))
            gen_imgs = generator.predict(noise)
            # Train the generator (wants discriminator to mistake images as real)
            g_loss = GANModel.train_on_batch(noise, valid)
            
            current_generator_accuracy = g_loss[1]
            print ("%d [G loss: %.3f] [G acc: %.3f]" % (epoch, g_loss[0], g_loss[1]))
            
        GANModel.save(os.path.join(weights_folder, 'GANModel_'+str(epoch)+'_loss_'+str(g_loss[0])+'.h5'))
        discriminator.save(os.path.join(weights_folder, 'discriminator_'+str(epoch)+'_loss_'+str(d_loss)+'.h5'))
        generator.save(os.path.join(weights_folder, 'generator_'+str(epoch)+'_loss_'+str(g_loss[0])+'.h5'))
        
        # Save an example
        fig=plt.figure(figsize=(64, 64))
        plt.imshow(gen_imgs[0, :, :, 0], cmap='gray')
        plt.axis('off')
        plt.savefig(os.path.join(RUN_FOLDER, "output/"+str(epoch)+".png"), format='png')
        plt.close()
        
        # Discriminator too strong for Generator to be able to learn
        #if (d_loss < 0.05):
            #print('Discirminator too strong')
            #break
        # Plot the progress
        #if (epoch%10 == 0):
        print ("%d [D loss: (%.3f)(R %.3f, F %.3f)] [D acc: (%.3f)(%.3f, %.3f)] [G loss: %.3f] [G acc: %.3f]" % (epoch, d_loss, d_loss_real, d_loss_fake, d_acc, d_acc_real, d_acc_fake, g_loss[0], g_loss[1]))
        #else:
        #    print ("%d [G loss: %.3f] [G acc: %.3f]" % (epoch, g_loss[0], g_loss[1]))

        # If at save interval => save generated image samples
        #if epoch % save_interval == 0:
            ##self.save_imgs(epoch)
        
# Quelle(https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py)

In [14]:
# Training test --> Train generator until 100% then train discriminator until 50% Repeat
train_custom(1000, batch_size=32)

In [ ]:
fig = plt.figure()
plt.plot([x[0] for x in d_losses], color='black', linewidth=0.25)

plt.plot([x[1] for x in d_losses], color='green', linewidth=0.25)
plt.plot([x[2] for x in d_losses], color='red', linewidth=0.25)
plt.plot([x[0] for x in g_losses], color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

# plt.xlim(0, 2000)
# plt.ylim(0, 2)

plt.savefig(os.path.join(RUN_FOLDER, "output/loss_chart.png"), format='png')
plt.show()
plt.close

In [ ]:
batch_size=50
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

noise = np.random.normal(0, 1, (batch_size, z_dim))
generated_images = generator.predict(noise)
#print(discriminator.predict(generated_images))
print(GANModel.predict(noise))
#print(GANModel.predict(noise))
for i in range(generated_images.shape[0]):
    fig=plt.figure(figsize=(64, 64))
    plt.subplot(1, 8, i+1)
    plt.imshow(generated_images[i, :, :, 0], cmap='gray')
    plt.axis('off')

In [ ]:
reshapedPlot = reshaped[58].reshape(128,88)
for i in range(1):
    fig=plt.figure(figsize=(64, 64))
    plt.subplot(1, 8, i+1)
    plt.imshow(reshapedPlot, cmap='gray')
    plt.axis('off')
    #plt.close()

In [ ]:
GANModel_loaded = GANModel
discriminator_loaded = discriminator
generator_loaded = generator
GANModel_loaded.load_weights(os.path.join(weights_folder, 'weights-GANModel.h5'))
discriminator_loaded.load_weights(os.path.join(weights_folder, 'weights-discriminator.h5'))
generator_loaded.load_weights(os.path.join(weights_folder, 'weights-generator.h5'))

batch_size=10
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

noise = np.random.normal(0, 1, (batch_size, z_dim))
generated_images = generator_loaded.predict(noise)
#print(discriminator.predict(generated_images))
print(GANModel_loaded.predict(noise))
#print(GANModel.predict(noise))
for i in range(generated_images.shape[0]):
    fig=plt.figure(figsize=(64, 64))
    plt.subplot(1, 8, i+1)
    plt.imshow(generated_images[i, :, :, 0], cmap='gray')
    plt.axis('off')

In [ ]:
gen_img = generated_images[1].reshape(128, 88)
result = np.zeros((pianrollLength, 128))
result[:,lowestNotePossible:highestNotePossible] = gen_img
result.shape

# Test convert an image back to see the Splitted Arrays Plotted

In [ ]:
def generate_Midi_File(img):
    gen_img = img.reshape(128, 88)
    result = np.zeros((pianrollLength, 128))
    result[:,lowestNotePossible:highestNotePossible] = gen_img
    result.shape
    result = result * 255.
    track = Track(pianoroll=result, program=0, is_drum=False,name='my awesome piano')
    track.plot()
    multi = Multitrack()
    multi.tracks[0] = track
    multi.write('output/test_0002.mid')
    #multi.write('output/test_zebra.npz')

In [ ]:
generate_Midi_File(gen_img)

In [ ]:
plot_model(GANModel, to_file=os.path.join(RUN_FOLDER ,'images/GANModel.png'), show_shapes = True, show_layer_names = True)
plot_model(discriminator, to_file=os.path.join(RUN_FOLDER ,'images/discriminator.png'), show_shapes = True, show_layer_names = True)
plot_model(generator, to_file=os.path.join(RUN_FOLDER ,'images/generator.png'), show_shapes = True, show_layer_names = True)

In [ ]:
def save_model(run_folder):
    GANModel.save(os.path.join(RUN_FOLDER, 'images/GANModel.h5'))
    discriminator.save(os.path.join(RUN_FOLDER, 'images/discriminator.h5'))
    generator.save(os.path.join(RUN_FOLDER, 'images/generator.h5'))
    #pickle.dump(open( os.path.join(run_folder, "obj.pkl"), "wb" ))

In [ ]:
GANModel.save(os.path.join(RUN_FOLDER, 'images/GANModel.h5'))
discriminator.save(os.path.join(RUN_FOLDER, 'images/discriminator.h5'))
generator.save(os.path.join(RUN_FOLDER, 'images/generator.h5'))

In [ ]:
#GANModel.save(os.path.join(weights_folder, 'weights-okay-GANModel.h5'))
#discriminator.save(os.path.join(weights_folder, 'weights-okay-discriminator.h5'))
#generator.save(os.path.join(weights_folder, 'weights-okay-generator.h5'))

In [ ]:
def train_normal(epochs, batch_size=128):
    # Save the Generator and discriminator models
    #save_model(generator, os.path.join(RUN_FOLDER, 'images/generator'))
    #save_model(discriminator, os.path.join(RUN_FOLDER, 'images/discriminator'))
    #save_model(GANModel, os.path.join(RUN_FOLDER, r"images/GANModel"))
    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    best_loss = 0.5
    for epoch in range(epochs):
        
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        gen_imgs = generator.predict(noise)
        # Select a random half of images
        idx = np.random.randint(0, reshaped.shape[0], batch_size)
        imgs = reshaped[idx]

        # ---------------------
        #  Train Discriminator
        # ---------------------      
        d_loss_real, d_acc_real =  discriminator.train_on_batch(imgs, valid)
        d_loss_fake, d_acc_fake =  discriminator.train_on_batch(gen_imgs, fake)
        d_loss =  0.5 * (d_loss_real + d_loss_fake)
        d_acc = 0.5 * (d_acc_real + d_acc_fake)
        print ("%d [D loss: (%.3f)(R %.3f, F %.3f)] [D acc: (%.3f)(%.3f, %.3f)]" % (epoch, d_loss, d_loss_real, d_loss_fake, d_acc, d_acc_real, d_acc_fake))


        # ---------------------
        #  Train Generator
        # ---------------------
        g_loss = GANModel.train_on_batch(noise, valid)
        print ("%d [G loss: %.3f] [G acc: %.3f]" % (epoch, g_loss[0], g_loss[1]))
        
        # Save an example
        fig=plt.figure(figsize=(64, 64))
        plt.imshow(gen_imgs[0, :, :, 0], cmap='gray')
        plt.axis('off')
        plt.savefig(os.path.join(RUN_FOLDER, "images/"+str(epoch)+".png"), format='png')
        plt.close()
    
        print ("%d [D loss: (%.3f)(R %.3f, F %.3f)] [D acc: (%.3f)(%.3f, %.3f)] [G loss: %.3f] [G acc: %.3f]" % (epoch, d_loss, d_loss_real, d_loss_fake, d_acc, d_acc_real, d_acc_fake, g_loss[0], g_loss[1]))
        
# Quelle(https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py)

In [ ]:
d_losses = np.load(os.path.join(RUN_FOLDER, "D_loss.npy"))
g_losses = np.load(os.path.join(RUN_FOLDER, "G_loss.npy"))